In [ ]:
import numpy as np
import pandas as pd
import keras
import cv2
from matplotlib import pyplot as plt
import os
import random
from PIL import Image


In [ ]:
df= pd.read_csv("train.csv")
base_path="./images/"

In [ ]:
df

In [ ]:
samples= 20000
df=df.loc[df["id"].str.startswith('00',na=False),:]
num_classes=len(df["landmark_id"].unique())
num_data=len(df)

In [ ]:
num_classes

In [ ]:
num_data

In [ ]:
data = pd.DataFrame(df['landmark_id'].value_counts())
data.reset_index(inplace=True)
data.head()

In [ ]:
data.tail()

In [ ]:
data.coloumn=['landmark_id','count']

In [ ]:
data['count'].describe()

In [ ]:
plt.hist(data['count'],100,range=(0,270),label="test")

In [ ]:
data['count'].between(0,5).sum()

In [ ]:
data['count'].between(5,10).sum()

In [ ]:
plt.hist(df["landmark_id"],bins=df["landmark_id"].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder
lencoder=LabelEncoder()
lencoder.fit(df["landmark_id"])

In [ ]:
df.head()

In [ ]:
def encode_label(lbl):
    return lencoder.transform(lbl)

In [ ]:
def decode_label(lbl):
    return lencoder.inverse_transform(lbl)

In [ ]:
def get_image_from_number(num,df):
    fname, label = df.iloc[num,:]
    fname=fname+'.jpg'
    f1=fname[0]
    f2=fname[1]
    f3=fname[2]
    path=os.path.join(f1,f2,f3,fname)
    im=cv2.imread(os.path.join(base_path,path))
    return im,label

In [ ]:
print("4 Sample images from random classes")
fig=plt.figure(figsize=(16,16))
for i in range(1,5):
  ri= random.choices(os.listdir(base_path),k=3)
  folder=base_path+'/'+ri[0]+'/'+ri[1]+'/'+ri[2]
  random_img= random.choices(os.listdir(folder))
  img=np.array(Image.open(folder+'/'+random_img))
  fig.add_subplot(1,4,i)
  plt.imshow(img)
  plt.axis('off')
plt.show()


In [ ]:
from keras.applications.vgg19 import VGG19
from keras.layers import *
from keras import Sequential
import tensorflow as tf
tf.compat.v1.disable_eager_execution()



In [ ]:
learning_rate=0.0001
decay_speed=1e-6
momentum=0.09
loss_function="sparse_categorial_crossentropy"
source_model=VGG19(weights=None)
drop_layer= Dropout(0.5)
drop_layer2=Dropout(0.5)

In [ ]:
model=Sequential()
for layer in source_model.layers[:-1]:
    if layer==source_model.layers[-25]:
       model.add(BatchNormalization())
    model.add(layer)
    model.add(Dense(num_classes,activation="softmax"))
    model.summary()

In [ ]:
optim1 =keras.optimizers.RMSprop(lr=learning_rate)

model.compile(optimizer=optim1,
              loss=loss_function,
              metrics=["accuracy"])

In [ ]:
def image_reshape(im,target_size):
    return cv2.resize(im,target_size)

In [ ]:
def get_batch(dataframe,start,batch_size):
    image_array=[]
    label_array=[]

    end_img=start+batch_size
    if(end_img)>len(dataframe):
        end_img=len(dataframe)

    for idx in range (start,end_img):
        n=idx
        im,label=get_image_from_number(n,dataframe)
        im=image_reshape(im(224,224))/255.0
        image_array.append(im)
        label_array.append(label)
    label_array=encode_label(label,array)
    return np.array(imay=ge_array),np.array(label_array)





In [ ]:
batch_size=16
epoch_shuffle=True
weight_classes=True
epochs=1

#split
train,val=np.split(df.sample(frac=1),[int(0.8*len(df))])
print(len(train))
print(len(val))

In [ ]:
for e in range(epochs):
    if epoch_shuffle:
        train = train.sample(frac=1)
    for it in range(int(np.ceil(len(train) / batch_size))):
        X_train, Y_train = get_batch(train, it*batch_size, batch_size)
        model.train_on_batch(X_train, Y_train)

model.save("Model")

In [ ]:
batch_size = 16
errors = 0
good_preds=[]
bad_preds=[]

for it in range(int(np.ceil(len(val)/batch_size))):
    X_val,Y_val =get_batch(val,it*batch_size,batch_size)
    result=model.predict(X_val)
    cla=np.argmax(result,axis=1)
    for idx, res in enumerate(result):
        if cla[idx]!= Y_val[idx]:
           errors =errors+1
           bad_preds.append([batch_size*it+idx,cla[idx],res[cla[idx]]])
        else:
             good_preds.append([batch_size*it+idx, cla[idx], res[cla[idx]]])


In [ ]:
good_preds=np.array(good_preds)
good_preds=np.array(sorted(good_preds,key= lambda x:x[2], reverse=True))


In [ ]:
fig=plt.figure(figsize=(16,16))
for i in range (1,6):
    n=int(good_preds[i,0])
    img,lbl=get_image_from_number(n,val)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    fig.add_subplot(1,5,i)
    plt.imshow(img)
    lbl2=np.array(int(good_preds[i,1])).reshape(1,1)
    sample_cnt=list(df.landmark_id).count(lbl)

    plt.tittle("Label: "+str(lbl)+"\ nClassfield as:"+str(decode_label(lbl)+ "\n Samples in class"+ str(lbl)+":"))
    plt.axs('off')
plt.show()